In [3]:
!pip install yandex_cloud_ml_sdk


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.3/138.3 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 44.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 105.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 95.7 MB/s eta 0:00:00
  Attempting uninstall: cryptography
    Found existing installation: cryptography 43.0.3
    Uninstalling cryptography-43.0.3:
      Successfully uninstalled cryptography-43.0.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pydrive2 1.21.3 requires cryptography<44, but you have cryptography 44.0.3 which is incompatible.
pyopenssl 24.2.1 requires cryptography<44,>=41.0.5, but you have cryptography 44.0.3 which is incompatible.


In [4]:
from __future__ import annotations
from yandex_cloud_ml_sdk import YCloudML

In [5]:
API_KEY   = 'ur api key'
FOLDER_ID = 'ur folder id'
URL       = "ur url"


In [6]:
model_uri = f"gpt://{FOLDER_ID}/yandexgpt/latest"
MODEL_URI = f"gpt://{FOLDER_ID}/yandexgpt/latest"

In [7]:
import os

In [8]:
!git clone https://github.com/Enforcer/clean-architecture


Cloning into 'clean-architecture'...
remote: Enumerating objects: 3681, done.
remote: Counting objects: 100% (503/503), done.
remote: Compressing objects: 100% (149/149), done.
remote: Total 3681 (delta 409), reused 354 (delta 354), pack-reused 3178 (from 1)
Receiving objects: 100% (3681/3681), 1.15 MiB | 4.70 MiB/s, done.
Resolving deltas: 100% (2181/2181), done.


In [22]:
from pathlib import Path

def auto_find_code_root(repo_root: Path) -> Path:


    best, max_subs = None, 0
    for sub in repo_root.iterdir():
        if not sub.is_dir():
            continue
        cnt = sum(1 for _ in sub.iterdir() if _.is_dir())
        if cnt > max_subs:
            best, max_subs = sub, cnt
    if best is None:
        raise FileNotFoundError(f"Не нашёл ни одной папки в {repo_root}")
    return best


repo_root = Path("/content/clean-architecture")
code_root = auto_find_code_root(repo_root)
docs_root = Path("/content/generated_docs") / code_root.name

print("▶ Ищем код в:", code_root)
print("▶ Документации будут в:", docs_root)


▶ Ищем код в: /content/clean-architecture/auctioning_platform
▶ Документации будут в: /content/generated_docs/auctioning_platform


In [10]:
import os

output_dir = code_root
os.makedirs(output_dir, exist_ok=True)


In [11]:
from pathlib import Path
import os

def collect_python_files(repo_path: str) -> list[str]:
    """
      собирает все .py-файлы
    """
    py_files = []
    for root, _, files in os.walk(repo_path):
        for fn in files:
            if fn.endswith(".py"):
                py_files.append(os.path.join(root, fn))
    return py_files

files = collect_python_files(repo_root)
print("Найдено .py-файлов:", len(files))


Найдено .py-файлов: 141


 ### Документация файлов

In [12]:
import requests, json
from pathlib import Path

system_prompt = (
    "Ты — эксперт по архитектуре программного обеспечения. Должен в ответ на присланный код присылать краткое описание того, как работает эта программа. Не вставляй никакой код в свой ответ. Только краткое текстовое описание работы"
)

headers = {
    "Authorization": f"Api-Key {API_KEY}",
    "x-folder-id":   FOLDER_ID,
    "Content-Type":  "application/json"
}



def generate_doc_for_file(filepath: str) -> str:
    code = Path(filepath).read_text(encoding="utf-8")

    payload = {
        "modelUri": model_uri,
        "completionOptions": {
            "stream": False,
            "temperature": 0.3,
            "maxTokens": 1500
        },
        "messages": [
            {"role": "system", "text": system_prompt},
            {"role": "user",   "text": code}
        ]
    }

    r = requests.post(URL, headers=headers, json=payload, timeout=120)
    r.raise_for_status()
    return r.json()["result"]["alternatives"][0]["message"]["text"]


In [13]:

repo_path  = repo_root
output_dir = Path("/content/generated_docs")
output_dir.mkdir(parents=True, exist_ok=True)

py_files = collect_python_files(repo_path)
print(f"Будем обрабатывать {len(py_files)} файлов…")

for path in py_files:
    try:
        print("Обработка:", path)
        doc = generate_doc_for_file(path)
        md_name = Path(path).relative_to(repo_path).with_suffix(".md")
        out_path = output_dir / md_name
        out_path.parent.mkdir(parents=True, exist_ok=True)
        out_path.write_text(doc, encoding="utf-8")
        print("Сохранено:", out_path)
    except Exception as e:
        print("Ошибка для", path, ":", e)


Будем обрабатывать 141 файлов…
Обработка: /content/clean-architecture/auctioning_platform/payments/setup.py
Сохранено: /content/generated_docs/auctioning_platform/payments/setup.md
Обработка: /content/clean-architecture/auctioning_platform/payments/payments/dao.py
Сохранено: /content/generated_docs/auctioning_platform/payments/payments/dao.md
Обработка: /content/clean-architecture/auctioning_platform/payments/payments/models.py
Сохранено: /content/generated_docs/auctioning_platform/payments/payments/models.md
Обработка: /content/clean-architecture/auctioning_platform/payments/payments/events.py
Сохранено: /content/generated_docs/auctioning_platform/payments/payments/events.md
Обработка: /content/clean-architecture/auctioning_platform/payments/payments/__init__.py
Сохранено: /content/generated_docs/auctioning_platform/payments/payments/__init__.md
Обработка: /content/clean-architecture/auctioning_platform/payments/payments/facade.py
Сохранено: /content/generated_docs/auctioning_platform

In [ ]:
!zip -r /content/docs.zip /content/generated_docs


  adding: content/generated_docs/ (stored 0%)
  adding: content/generated_docs/auctioning_platform/ (stored 0%)
  adding: content/generated_docs/auctioning_platform/payments/ (stored 0%)
  adding: content/generated_docs/auctioning_platform/payments/payments/ (stored 0%)
  adding: content/generated_docs/auctioning_platform/payments/payments/tests/ (stored 0%)
  adding: content/generated_docs/auctioning_platform/payments/payments/tests/test_facade.md (deflated 62%)
  adding: content/generated_docs/auctioning_platform/payments/payments/tests/conftest.md (deflated 43%)
  adding: content/generated_docs/auctioning_platform/payments/payments/tests/api/ (stored 0%)
  adding: content/generated_docs/auctioning_platform/payments/payments/tests/api/test_consumer.md (deflated 51%)
  adding: content/generated_docs/auctioning_platform/payments/payments/dao.md (deflated 61%)
  adding: content/generated_docs/auctioning_platform/payments/payments/facade.md (deflated 64%)
  adding: content/generated_docs

In [14]:
!apt-get update -qq
!apt-get install -qq -y default-jre graphviz


W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Selecting previously unselected package default-jre-headless.
(Reading database ... 126101 files and directories currently installed.)
Preparing to unpack .../0-default-jre-headless_2%3a1.11-72build2_amd64.deb ...
Unpacking default-jre-headless (2:1.11-72build2) ...
Selecting previously unselected package libxtst6:amd64.
Preparing to unpack .../1-libxtst6_2%3a1.2.3-1build4_amd64.deb ...
Unpacking libxtst6:amd64 (2:1.2.3-1build4) ...
Selecting previously unselected package openjdk-11-jre:amd64.
Preparing to unpack .../2-openjdk-11-jre_11.0.26+4-1ubuntu1~22.04_amd64.deb ...
Unpacking openjdk-11-jre:amd64 (11.0.26+4-1ubuntu1~22.04) ...
Selecting previously unselected package default-jre.
Preparing to unpack .../3-default-jre_2%3a1.11-72build2_amd64.deb ...
Unpacking default-jre (2:1.11-72build2) ...
Sel

In [15]:
!wget -q https://github.com/plantuml/plantuml/releases/latest/download/plantuml.jar -O /content/plantuml.jar


# work


### генерация докментаций для модулей

In [17]:
HEADERS = {
    "Authorization": f"Api-Key {API_KEY}",
    "x-folder-id":   FOLDER_ID,
    "Content-Type":  "application/json"
}

In [ ]:
docs

In [18]:
from pathlib import Path
import requests
import json


MAX_CHARS = 10000

def merge_batch(mod_name: str, files_batch: list[tuple[str, str]]) -> str:
    joined = "\n\n".join(f"### {Path(path).name}\n{text}" for path, text in files_batch)
    prompt = (
        f"Документации файлов модуля `{mod_name}`. Слей их в единое описание. "
        f"Опиши архитектуру, назначение, компоненты и use-cases. Markdown-формат.\n\n{joined}"
    )
    payload = {
        "modelUri": MODEL_URI,
        "completionOptions": {"stream": False, "temperature": 0.2, "maxTokens": 1200},
        "messages": [
            {"role": "system", "text": "Ты — AI-документовед. Объединяй документации файлов в описание модуля."},
            {"role": "user", "text": prompt}
        ]
    }
    response = requests.post(URL, headers=HEADERS, json=payload, timeout=120)
    response.raise_for_status()
    return response.json()["result"]["alternatives"][0]["message"]["text"]

def split_into_batches(md_files, max_chars=MAX_CHARS):
    batches, batch, size = [], [], 0
    for f in md_files:
        length = len(f[1])
        if size + length > max_chars and batch:
            batches.append(batch)
            batch, size = [], 0
        batch.append(f)
        size += length
    if batch:
        batches.append(batch)
    return batches

for mod_dir in docs_root.iterdir():
    if not mod_dir.is_dir():
        continue
    mod_name = mod_dir.name
    print(f"\nМодуль: {mod_name}")

    md_files = [(str(f), f.read_text(encoding="utf-8")) for f in mod_dir.rglob("*.md")]
    if not md_files:
        print(f"Нет документов для модуля {mod_name}")
        continue

    batches = split_into_batches(md_files)
    print(f"Разбили на {len(batches)} батчей.")

    module_docs = []
    for i, batch in enumerate(batches, 1):
        print(f"🔸 Отправляем батч {i}/{len(batches)} (файлов: {len(batch)})")
        try:
            part_doc = merge_batch(mod_name, batch)
            module_docs.append(part_doc)
        except requests.exceptions.HTTPError as e:
            print(f"Ошибка на батче {i} модуля '{mod_name}': {e}")
            continue

    final_doc = "\n\n".join(module_docs)

    #  Теперь сохраняем внутри папки модуля:
    out_path = mod_dir / f"{mod_name}_module.md"
    out_path.write_text(final_doc, encoding="utf-8")
    print(f"Сохранено в папке модуля: {out_path}")



📦 Модуль: payments
Разбили на 1 батчей.
🔸 Отправляем батч 1/1 (файлов: 15)
Сохранено в папке модуля: /content/generated_docs/auctioning_platform/payments/payments_module.md

📦 Модуль: web_app
Разбили на 1 батчей.
🔸 Отправляем батч 1/1 (файлов: 11)
Сохранено в папке модуля: /content/generated_docs/auctioning_platform/web_app/web_app_module.md

📦 Модуль: auctions_infrastructure
Разбили на 1 батчей.
🔸 Отправляем батч 1/1 (файлов: 10)
Сохранено в папке модуля: /content/generated_docs/auctioning_platform/auctions_infrastructure/auctions_infrastructure_module.md

📦 Модуль: web_app_models
Разбили на 1 батчей.
🔸 Отправляем батч 1/1 (файлов: 2)
Сохранено в папке модуля: /content/generated_docs/auctioning_platform/web_app_models/web_app_models_module.md

📦 Модуль: auctions
Разбили на 2 батчей.
🔸 Отправляем батч 1/2 (файлов: 17)
🔸 Отправляем батч 2/2 (файлов: 8)
Сохранено в папке модуля: /content/generated_docs/auctioning_platform/auctions/auctions_module.md

📦 Модуль: customer_relationship
Разб

### Документация проекта

In [19]:
from pathlib import Path
import requests
import json

output_path = docs_root / "project_overview.md"


from urllib.parse import quote

project_root_path = repo_root

module_entries = []
module_docs = []

for f in docs_root.rglob("*_module.md"):
    mod_name = f.stem.replace("_module", "")
    rel_doc_path = f.relative_to(docs_root)

    source_mod_path = project_root_path / rel_doc_path.parent
    source_mod_path_str = str(source_mod_path).replace("\\", "/")


    md_link = f"[{mod_name}]({quote(source_mod_path_str)})"

    module_entries.append(f"- {md_link}  \n  📁 Путь: `{source_mod_path_str}`")

    text = f.read_text(encoding="utf-8")
    module_docs.append((mod_name, text))

module_summary_md = "### Основные компоненты и модули\n\n" + "\n".join(module_entries)



joined = "\n\n".join(f"## {mod_name}\n\n{text[:2000]}" for mod_name, text in module_docs)



prompt = (
    "Создай краткое и структурированное описание проекта на основе документации его модулей. "
    "Включи следующие разделы:\n"
    "- Назначение системы\n"
    "- Архитектурный стиль и принципы\n"
    "- Основные компоненты и модули (с названиями, ссылками и путями)\n"
    "- Взаимодействие и ключевые сценарии (use-cases)\n\n"
    "Ответ — в Markdown.\n\n"
    f"{module_summary_md}\n\n"
    f"{joined}"
)


payload = {
    "modelUri": MODEL_URI,
    "completionOptions": {"stream": False, "temperature": 0.2, "maxTokens": 2000},
    "messages": [
        {"role": "system", "text": "Ты — AI-архитектор. Составь краткое описание проекта на основе модульной документации."},
        {"role": "user", "text": prompt}
    ]
}


response = requests.post(URL, headers=HEADERS, json=payload, timeout=120)
response.raise_for_status()
overview = response.json()["result"]["alternatives"][0]["message"]["text"]

output_path.write_text(overview, encoding="utf-8")
print(f"Сохранено: {output_path}")


Сохранено: /content/generated_docs/auctioning_platform/project_overview.md


In [20]:

import subprocess, shutil
from pathlib import Path

project_root = code_root


subprocess.run(["pip", "install", "-q", "pylint", "graphviz"], check=True)


for mod_dir in project_root.iterdir():
    if not mod_dir.is_dir():
        continue
    name = mod_dir.name

    candidate = mod_dir / name
    if (candidate / "__init__.py").exists():
        target = candidate
    elif any(f.suffix == ".py" for f in mod_dir.iterdir()):
        target = mod_dir
    else:
        print(f"Пропускаем {name}: не является пакетом")
        continue


    total_classes = sum(f.read_text(encoding="utf-8").count("class ")
                        for f in target.rglob("*.py"))
    if total_classes == 0:
        print(f"– {name}: нет классов, пропускаем")
        continue

    print(f"Обрабатываем {name} ({total_classes} классов)")

    try:

        subprocess.run(
            ["pyreverse", "-o", "dot", "-p", name, str(target)],
            check=True, stderr=subprocess.PIPE
        )
        dot_file = Path(f"classes_{name}.dot")
        if not dot_file.exists():
            print(f".dot не найден для {name}")
            continue

        lines = dot_file.read_text(encoding="utf-8").splitlines()
        links = [l for l in lines if "->" in l and not l.strip().startswith("//")]
        if not links:
            print(f"Нет связей в {name}, пропускаем")
            dot_file.unlink()
            continue

        involved = set()
        for link in links:
            a, b = link.split("->", 1)
            involved.add(a.strip())
            involved.add(b.strip())

        new_lines = []
        inside = False
        for l in lines:
            s = l.strip()
            if s.startswith("digraph"):
                inside = True
                new_lines.append(l)
            elif s == "}":
                inside = False
                new_lines.append(l)
            elif inside:
                if "->" in l:
                    new_lines.append(l)
                elif "[" in l and "]" in l:
                    node = l.split("[", 1)[0].strip()
                    if node in involved:
                        new_lines.append(l)
            else:
                new_lines.append(l)

        dot_file.write_text("\n".join(new_lines), encoding="utf-8")

        mod_docs_dir = docs_root / name
        mod_docs_dir.mkdir(parents=True, exist_ok=True)

        out_png = mod_docs_dir / f"{name}_diagram.png"
        subprocess.run([
            "dot", "-Tpng", str(dot_file), "-o", str(out_png)
        ], check=True, stderr=subprocess.PIPE)
        print(f"Сохранено: {out_png.relative_to(docs_root.parent)}")


        dot_file.unlink()

    except subprocess.CalledProcessError as e:
        msg = e.stderr.decode().splitlines()[-1] if e.stderr else str(e)
        print(f"Ошибка для {name}: {msg}")


Обрабатываем payments (18 классов)
Сохранено: auctioning_platform/payments/payments_diagram.png
Обрабатываем web_app (10 классов)
Нет связей в web_app, пропускаем
Обрабатываем auctions_infrastructure (6 классов)
Сохранено: auctioning_platform/auctions_infrastructure/auctions_infrastructure_diagram.png
Обрабатываем web_app_models (3 классов)
Нет связей в web_app_models, пропускаем
Обрабатываем auctions (29 классов)
Сохранено: auctioning_platform/auctions/auctions_diagram.png
Обрабатываем customer_relationship (10 классов)
Сохранено: auctioning_platform/customer_relationship/customer_relationship_diagram.png
Обрабатываем foundation (17 классов)
Сохранено: auctioning_platform/foundation/foundation_diagram.png
Обрабатываем db_infrastructure (1 классов)
Нет связей в db_infrastructure, пропускаем
Обрабатываем main (8 классов)
Нет связей в main, пропускаем
Обрабатываем shipping_infrastructure (2 классов)
Нет связей в shipping_infrastructure, пропускаем
Обрабатываем processes (8 классов)
Сохра